In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import make_pipeline

In [2]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


### Логистическая регрессия

In [4]:
# Выбираем параметры модели и делаем их предобработку

selectedColumns = data[['age', 'education', 'gender',  'race', 'hours-per-week']]
X = pd.get_dummies(selectedColumns, columns = ['education', 'gender',  'race'])
X.head()

,age,hours-per-week,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,...,education_Preschool,education_Prof-school,education_Some-college,gender_Female,gender_Male,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,40,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,38,50,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,28,40,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,44,40,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4,18,30,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1


In [5]:
# Обработка целевого значения

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit( data['income'] )

LabelEncoder()

In [6]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [7]:
le.inverse_transform([1, 0, 1])

array(['>50K', '<=50K', '>50K'], dtype=object)

In [8]:
y = pd.Series (data = le.transform(data['income']))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [9]:
from sklearn.pipeline import make_pipeline                # Используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# Обучаем на части датасета (train)

model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [12]:
predictions[:20] # Посмотрим как выглядят наши предсказания

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
model.predict(X_test) # Получаем массив

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
# Получаем наш скор (точность предсказания) на обучающей и тестовой выборках.

model.score(X_train, y_train) 

0.8007546286817397

In [15]:
model.score(X_test,y_test)

0.8024977820241589

### SVM

In [16]:
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) # Используем пайплайны
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [17]:
# Снова получаем наш скор, для обучающей и тестовой выборок

clf.score(X_train, y_train)

0.8064582175553541

In [18]:
clf.score(X_test, y_test)

0.8074114515798813

In [19]:
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto')

In [20]:
svc.score(X_train, y_train)

0.8076574336774986

In [21]:
svc.score(X_test, y_test)

0.8065242612434314